In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib
import re

import tools

In [2]:
df = pd.read_csv("../data/ga000v0001.csv", header=0)
df.head(10)
col_list = df.columns

In [3]:
col_list

Index(['国・地域', '地域階層', '地域コード', '国・地域(人)', '昭和54年', '昭和55年', '昭和56年', '昭和57年',
       '昭和58年', '昭和59年', '昭和60年', '昭和61年', '昭和62年', '昭和63年', '昭和64年', '平成2年',
       '平成3年', '平成4年', '平成5年', '平成6年', '平成7年', '平成8年', '平成9年', '平成10年',
       '平成11年', '平成12年', '平成13年', '平成14年', '平成15年', '平成16年', '平成17年', '平成18年',
       '平成19年', '平成20年', '平成21年', '平成22年', '平成23年', '平成24年', '平成25年', '平成26年',
       '平成27年', '平成28年', '平成29年', '平成30年', '平成31年', '令和2年', '令和3年', '令和4年',
       '令和5年', '令和6年', '令和7年'],
      dtype='object')

In [4]:
df["国_地域"] = df["国・地域"]+"_"+df["国・地域(人)"]

In [5]:
df_value = ['昭和54年', '昭和55年', '昭和56年', '昭和57年',
       '昭和58年', '昭和59年', '昭和60年', '昭和61年', '昭和62年', '昭和63年', '昭和64年', '平成2年',
       '平成3年', '平成4年', '平成5年', '平成6年', '平成7年', '平成8年', '平成9年', '平成10年',
       '平成11年', '平成12年', '平成13年', '平成14年', '平成15年', '平成16年', '平成17年', '平成18年',
       '平成19年', '平成20年', '平成21年', '平成22年', '平成23年', '平成24年', '平成25年', '平成26年',
       '平成27年', '平成28年', '平成29年', '平成30年', '平成31年', '令和2年', '令和3年', '令和4年',
       '令和5年', '令和6年', '令和7年']
df_new = pd.concat([df["国_地域"], df[df_value]], axis=1)
df_new = df_new.drop(index=[i for i in range(1512, 1530)])

In [6]:
df_tsa = df_new.set_index("国_地域").T
df_tsa.columns.name = None

# 和暦 index → 西暦 index へ変換
years = df_tsa.index.to_series().apply(tools.wareki_to_seireki)
df_tsa.index = pd.to_datetime(years, format="%Y")
df_tsa.index.name = "year"

df_tsa.head(3)

,総数_総数,総数_区部,総数_千代田区,総数_中央区,総数_港区,総数_新宿区,総数_文京区,総数_台東区,総数_墨田区,総数_江東区,...,その他_郡部,その他_瑞穂町,その他_日の出町,その他_檜原村,その他_奥多摩町,その他_島部,その他_大島支庁,その他_三宅支庁,その他_八丈支庁,その他_小笠原支庁
year,,,,,,,,,,,,,,,,,,,,,
1979-01-01,110862.0,93328.0,3188.0,459.0,7563.0,5488.0,2669.0,3098.0,1821.0,3653.0,...,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0
1980-01-01,112523.0,94819.0,3309.0,470.0,7718.0,6036.0,2745.0,3060.0,1849.0,3691.0,...,1.0,0.0,0.0,0.0,1.0,3.0,3.0,0.0,0.0,0.0
1981-01-01,114449.0,96434.0,3328.0,441.0,8258.0,6138.0,2825.0,3054.0,1889.0,3880.0,...,2.0,1.0,0.0,0.0,1.0,3.0,3.0,0.0,0.0,0.0


In [7]:
for col_name in df_tsa.columns:
    if df_tsa[col_name].dtype == "object":
        df_tsa[col_name] = df_tsa[col_name].astype(str)
        df_tsa[col_name] = df_tsa[col_name].str.replace(',', '', regex=False)
        df_tsa[col_name] = pd.to_numeric(df_tsa[col_name], errors='coerce')
    # print(f"{col_name}: {df_tsa[col_name].dtype}")

In [8]:
df_tsa.to_csv("../data/tsa_外国人人口.csv")